In [1]:
from pathlib import Path
import spikeinterface.extractors as si
import numpy as np
import pandas as pd
import scipy.io
import os
from numba import njit, prange
import pyarrow as pa
import pyarrow.parquet as pq
import hashlib


In [2]:
import numba

print("Numba threads:", numba.get_num_threads())  # see how many cores it will use
numba.set_num_threads(8)  # for example, if you want to cap or change it

Numba threads: 8


In [3]:
def safe_load(path):
    """Safely load .mat file, returning empty dict if file doesn't exist."""
    return scipy.io.loadmat(path) if Path(path).exists() else {}

def create_spike_summary(sort_path, electrode_num, patient, session):
    positive_file = sort_path / f"5.1/A_ss{electrode_num}_sorted_new.mat"
    negative_file = sort_path / f"5.2/A_ss{electrode_num}_sorted_new.mat"
    final_max_file = sort_path / f"final/A_ss{electrode_num}_Max_sorted_new.mat"
    final_min_file = sort_path / f"final/A_ss{electrode_num}_Min_sorted_new.mat"

    positive_spikes = safe_load(positive_file)
    negative_spikes = safe_load(negative_file)
    final_max_spikes = safe_load(final_max_file)
    final_min_spikes = safe_load(final_min_file)

    spike_summary = {
        "patient": patient,
        "session": session,
        "channel": electrode_num,
        "positive_data": {
            "positive_spikes": positive_spikes.get("useNegative", np.array([])).T,
            "positive_merged": positive_spikes.get("useNegativeMerged", np.array([])),
        },
        "negative_data": {
            "negative_spikes": negative_spikes.get("useNegative", np.array([])).T,
            "negative_merged": negative_spikes.get("useNegativeMerged", np.array([])),
        },
        "final_max": {
            "included": final_max_spikes.get("useNegative", np.array([])),
            "excluded": final_max_spikes.get("useNegativeExcluded", np.array([])).T,
        },
        "final_min": {
            "included": final_min_spikes.get("useNegative", np.array([])),
            "excluded": final_min_spikes.get("useNegativeExcluded", np.array([])).T,
        },
    }

    patient_data = (spike_summary["patient"], spike_summary["session"], spike_summary["channel"])

    good_units = {
        "positive_included": spike_summary["final_max"]["included"].tolist(),
        "negative_included": spike_summary["final_min"]["included"].tolist()
    }

    noise = {
        "positive_noise": [],
        "negative_noise": []
    }

    # Account for case where there is final max but no final min, or vice versa
    if spike_summary["final_max"]["included"].size != 0:
        noise["positive_noise"] = spike_summary["final_max"]["excluded"].tolist()
    else:
        noise["positive_noise"] = spike_summary["positive_data"]["positive_spikes"].tolist()
    
    if spike_summary["final_min"]["included"].size != 0:
        noise["negative_noise"] = spike_summary["final_min"]["excluded"].tolist()
    else:
        noise["negative_noise"] = spike_summary["negative_data"]["negative_spikes"].tolist()

    return patient_data, good_units, noise

In [4]:
@njit(parallel=True)
def _select_waveforms_for_label(labels, waveforms, label):
    n = labels.shape[0]
    if n == 0:
        return np.empty((0, 0), np.float32), 0

    n_t = waveforms.shape[1]

    # compute match mask (serial, cheap)
    mask = np.zeros(n, np.uint8)
    for i in range(n):
        if labels[i] == label and labels[i] != 0:
            mask[i] = 1

    # prefix positions for each matching spike
    pos = np.empty(n, np.int64)
    count = 0
    for i in range(n):
        pos[i] = count
        if mask[i]:
            count += 1

    out = np.empty((count, n_t), np.float32)

    # parallel copy of selected waveforms
    for i in prange(n):
        if mask[i]:
            out[pos[i], :] = waveforms[i]

    return out, count


def create_bag_of_spikes(patient_data,
                         spike_label,
                         alignment,
                         spike_type,
                         labels,
                         waveforms):
    labels = np.asarray(labels, dtype=np.int32)
    waveforms = np.asarray(waveforms, dtype=np.float32)

    if spike_label is None or labels.size == 0 or waveforms.size == 0:
        n_t = waveforms.shape[1] if waveforms.ndim == 2 else 0
        waveforms_for_label = np.empty((0, n_t), dtype=np.float32)
        n_selected = 0
    else:
        waveforms_for_label, n_selected = _select_waveforms_for_label(
            labels, waveforms, int(spike_label)
        )

    spike_dict = {
        "patient_data": patient_data,
        "spike_label": int(spike_label) if spike_label is not None else -1,
        "number_of_spikes": int(n_selected),
        "spike_type": spike_type,
        "alignment": alignment,
        "spikes": waveforms_for_label,
    }

    return spike_dict


def load_spike_arrays(sort_path, electrode_num):
    """Load label and waveform arrays once per channel to avoid repeated .mat I/O."""
    positive_file = sort_path / f"5.1/A_ss{electrode_num}_sorted_new.mat"
    negative_file = sort_path / f"5.2/A_ss{electrode_num}_sorted_new.mat"

    positive_spikes = safe_load(positive_file)
    negative_spikes = safe_load(negative_file)

    pos_labels = np.array(positive_spikes.get("assignedNegative", np.array([]))).ravel()
    pos_waves = np.array(positive_spikes.get("allSpikesCorrFree", np.array([])))

    neg_labels = np.array(negative_spikes.get("assignedNegative", np.array([]))).ravel()
    neg_waves = np.array(negative_spikes.get("allSpikesCorrFree", np.array([])))

    return {
        "positive": (pos_labels, pos_waves),
        "negative": (neg_labels, neg_waves),
    }


In [5]:
def list_subdirectories(dir_path, print_count = True):
    file_list = [name for name in os.listdir(dir_path) if os.path.isdir(os.path.join(dir_path, name))]
    if print_count == False:
        return file_list
    else:
        print(dir_path, "contains", file_list)
        return file_list


In [6]:
def get_units(good_units: dict,
              noise: dict,
              alignment: str,
              spike_type: str):
    # pick source dict
    src = good_units if spike_type == "good_units" else noise

    # key map - updated to match new snake_case keys
    key_map = {
        ("positive", "good_units"): "positive_included",
        ("negative", "good_units"): "negative_included",
        ("positive", "noise"): "positive_noise",
        ("negative", "noise"): "negative_noise",
    }

    key = key_map.get((alignment, spike_type), None)
    if key is None:
        return []

    arr = src.get(key, [])
    if isinstance(arr, (list, np.ndarray)) and len(arr) > 0:
        return arr[0]
    return []


def create_bag_of_bags(num_channels, patient_path, patient):
    base_path = Path(patient_path).parent
    alignments = ("positive", "negative")
    spike_types = ("noise", "good_units")

    sessions = list_subdirectories(patient_path)

    bag_of_bags = []

    for session_dir in sessions:
        sort_path = base_path / patient / session_dir / "sort"
        for channel in range(int(num_channels)):
            patient_data, good_units, noise = create_spike_summary(
                sort_path=sort_path,
                electrode_num=channel,
                patient=patient,
                session=session_dir,
            )

            # Load spike label/waveform arrays once per channel
            spike_data = load_spike_arrays(sort_path, channel)

            for spike_type in spike_types:
                for alignment in alignments:
                    labels, waveforms = spike_data[alignment]
                    units = np.asarray(
                        get_units(good_units, noise, alignment, spike_type),
                        dtype=np.int32,
                    )
                    for label in units:
                        bag = create_bag_of_spikes(
                            patient_data=patient_data,
                            spike_label=int(label),
                            alignment=alignment,
                            spike_type=spike_type,
                            labels=labels,
                            waveforms=waveforms,
                        )
                        bag_of_bags.append(bag)
                        print(
                            "Neuron: ",
                            bag["spike_label"],
                            " has ",
                            bag["number_of_spikes"],
                            "spikes",
                            f"Session={session_dir} ",
                            f"Channel={channel} ",
                        )
        print()

    return bag_of_bags        

In [14]:
def count_channels(patient_path):
    subdirectories = list_subdirectories(patient_path)
    raw_path = patient_path / subdirectories[0] / "raw/micros"
    # Count all files in raw_path that end with .ncs
    ncs_count = sum(1 for file in raw_path.iterdir() if file.is_file() and file.suffix == ".ncs")
    print(f"Number of .ncs files in {raw_path}: {ncs_count}")
    return str(ncs_count)

In [8]:
def make_unit_hash(patient: str, session: str, label_id: int) -> str:
    key = f"{patient}|{session}|{label_id}".encode("utf-8")
    # 4 bytes → 8 hex characters
    return hashlib.blake2b(key, digest_size=4).hexdigest()

In [9]:
def bags_and_spikes_from_list(bags):
    bag_rows = []
    spike_rows = []

    for bag_id, bag in enumerate(bags):
        patient, session, channel = bag["patient_data"]
        label_id = int(bag["spike_label"])
        spike_type = bag["spike_type"]
        alignment = bag["alignment"]
        waves = np.asarray(bag["spikes"], dtype="float32")
        n_spikes = waves.shape[0]
        mean_wave = waves.mean(axis=0) if n_spikes > 0 else np.zeros(256, np.float32)

        unit_id = make_unit_hash(patient, session, label_id)

        bag_rows.append(
            (bag_id, patient, session, int(channel), label_id,
             spike_type, alignment, n_spikes, mean_wave, unit_id)
        )

        for i in range(n_spikes):
            spike_rows.append((bag_id, i, waves[i], unit_id))  # optional: attach unit_id to each spike
    return bag_rows, spike_rows



In [26]:
def write_to_parquat(bag_rows, spike_rows, patient_path):
    bag_ids, patients, sessions, channels, labels, types, aligns, n_spikes, means, unit_ids = zip(*bag_rows)

    bags_table = pa.table({
        "bag_id":      pa.array(bag_ids, type=pa.int32()),
        "patient":     pa.array(patients),
        "session":     pa.array(sessions),
        "channel":     pa.array(channels, type=pa.int32()),
        "spike_label": pa.array(labels, type=pa.int32()),
        "spike_type":  pa.array(types),
        "alignment":   pa.array(aligns),
        "n_spikes":    pa.array(n_spikes, type=pa.int32()),
        "mean_waveform": pa.FixedSizeListArray.from_arrays(
            pa.array(np.stack(means).reshape(-1)), 256
        ),
        "unit_id": pa.array(unit_ids),
    })

    bag_id_s, spike_idxs, wf_list, spike_unit_ids = zip(*spike_rows)
    wf_stack = np.stack(wf_list).astype("float32")

    spikes_table = pa.table({
        "bag_id":   pa.array(bag_id_s, type=pa.int32()),
        "spike_idx": pa.array(spike_idxs, type=pa.int32()),
        "waveform": pa.FixedSizeListArray.from_arrays(
            pa.array(wf_stack.reshape(-1)), 256
        ),
        "unit_id": pa.array(spike_unit_ids),
    })

    bags_table_path = Path(patient_path) / "spike_bags.parquet"
    spikes_table_path = Path(patient_path) / "spikes.parquet"

    pq.write_table(bags_table, bags_table_path)
    pq.write_table(spikes_table, spikes_table_path)

    bags_df = pd.read_parquet(bags_table_path)
    spikes_df = pd.read_parquet(spikes_table_path)
    return bags_df, spikes_df

In [ ]:
base_path = "/Users/marco/Local_Sorting"
patient = "DA047 Sorted"
patient_path = Path(base_path) / patient
num_channels = count_channels(patient_path)
A047_Bag = create_bag_of_bags(num_channels, patient_path, patient)



/Users/marco/Local_Sorting/DA047 Sorted contains ['12-15', '12-13', '12-16']
Number of .ncs files in /Users/marco/Local_Sorting/DA047 Sorted/12-15/raw/micros: 16
/Users/marco/Local_Sorting/DA047 Sorted contains ['12-15', '12-13', '12-16']
Neuron:  1022  has  63 spikes Session=12-15  Channel=1 
Neuron:  1023  has  69 spikes Session=12-15  Channel=1 
Neuron:  1170  has  101 spikes Session=12-15  Channel=1 
Neuron:  1193  has  115 spikes Session=12-15  Channel=1 
Neuron:  1176  has  157 spikes Session=12-15  Channel=1 
Neuron:  1224  has  230 spikes Session=12-15  Channel=1 
Neuron:  1220  has  295 spikes Session=12-15  Channel=1 
Neuron:  964  has  429 spikes Session=12-15  Channel=1 
Neuron:  1213  has  1011 spikes Session=12-15  Channel=1 
Neuron:  872  has  63 spikes Session=12-15  Channel=1 
Neuron:  1078  has  177 spikes Session=12-15  Channel=1 
Neuron:  1110  has  529 spikes Session=12-15  Channel=1 
Neuron:  1101  has  558 spikes Session=12-15  Channel=1 
Neuron:  1081  has  1151

TypeError: cannot unpack non-iterable NoneType object

In [27]:
bag_rows, spike_rows = bags_and_spikes_from_list(A047_Bag)
bags_df, spike_df = write_to_parquat(bag_rows, spike_rows, patient_path)

In [30]:
spike_df

,bag_id,spike_idx,waveform,unit_id
0,0,0,"[-0.0019803504, -0.0019803504, -0.0019803504, ...",dab0693d
1,0,1,"[0.033008628, 0.033008628, 0.033008628, 0.0330...",dab0693d
2,0,2,"[1.1332413, 1.1332413, 1.1332413, 1.1332413, 1...",dab0693d
3,0,3,"[0.6874671, 0.6874671, 0.6874671, 0.6874671, 0...",dab0693d
4,0,4,"[0.7401716, 0.7401716, 0.7401716, 0.7401716, 0...",dab0693d
...,...,...,...,...
2509714,1128,12688,"[-0.03371385, -0.03371385, -0.03371385, -0.033...",3686eb77
2509715,1128,12689,"[-0.4289833, -0.4289833, -0.4289833, -0.428983...",3686eb77
2509716,1128,12690,"[-0.8535344, -0.8535344, -0.8535344, -0.853534...",3686eb77
2509717,1128,12691,"[0.8140469, 0.8140469, 0.8140469, 0.8140469, 0...",3686eb77
